# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234572662706                   -0.50    8.0
  2   -7.250302989241       -1.80       -1.40    1.0
  3   -7.251325823309       -2.99       -2.17    4.0
  4   -7.251324471946   +   -5.87       -2.48    3.0
  5   -7.251335309406       -4.97       -2.85    2.0
  6   -7.251337878402       -5.59       -3.18    2.0
  7   -7.251338776380       -6.05       -3.97    1.0
  8   -7.251338794222       -7.75       -4.25    3.0
  9   -7.251338798433       -8.38       -4.91    2.0
 10   -7.251338798697       -9.58       -5.59    3.0
 11   -7.251338798703      -11.27       -5.87    3.0
 12   -7.251338798704      -11.81       -6.32    2.0
 13   -7.251338798705      -12.65       -7.02    3.0
 14   -7.251338798705      -14.15       -7.38    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0814085699597854
[ Info: Arnoldi iteration step 2: normres = 0.3248894243097996
[ Info: Arnoldi iteration step 3: normres = 0.9586607252917054
[ Info: Arnoldi iteration step 4: normres = 0.39483078218914025
[ Info: Arnoldi iteration step 5: normres = 0.5802648315718634
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.85e-02, 7.18e-02, 4.84e-01, 2.96e-01, 1.46e-02)
[ Info: Arnoldi iteration step 6: normres = 0.1953226252577697
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.54e-02, 6.96e-02, 1.44e-01, 8.16e-02, 3.83e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07556894438005497
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (5.35e-04, 4.55e-03, 9.02e-03, 2.52e-02, 6.19e-02)
[ Info: Arnoldi iteration step 8: normres = 0.1030799578283486
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.33e-05